## Docker Compose Grid Domain and Network 🐋🎼

<img src="../../../docs/img/pygrid_logo.png" align="center"/>

Install Docker and docker-compose for your operating system: https://www.docker.com/

In [ ]:
# if you need syft you can try installing the latest release candidate with:
# !pip install syft==0.5.0

In [ ]:
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient
from syft import logger
import syft as sy
import torch as th

logger.remove()

# Settings

## Run the Compose File

```
$ cd PySyft
$ docker-compose -f docker/docker-compose.yml up -d
```

Run `$ docker ps` and check the output:
```
CONTAINER ID   IMAGE                           COMMAND                  CREATED          STATUS          PORTS                                       NAMES
28e7f96f410f   openmined/grid-network:latest   "bash -c 'cd /app &&…"   36 seconds ago   Up 32 seconds   0.0.0.0:5001->5000/tcp, :::5001->5000/tcp   network
5550f396ba64   openmined/grid-domain:latest    "bash -c 'cd /app &&…"   38 seconds ago   Up 33 seconds   0.0.0.0:5002->5000/tcp, :::5002->5000/tcp   domain
a41148d6806c   postgres:12                     "docker-entrypoint.s…"   41 seconds ago   Up 37 seconds   0.0.0.0:5434->5432/tcp, :::5434->5432/tcp   db.domain
f27e513d7c12   postgres:12                     "docker-entrypoint.s…"   41 seconds ago   Up 36 seconds   0.0.0.0:5433->5432/tcp, :::5433->5432/tcp   db.network```

In [ ]:
HOST_MAPPINGS = {}

In [ ]:
# We need to connect to our docker network using localhost and the published ports

In [ ]:
NETWORK_URL = "http://localhost:5001"
DOMAIN_URL = "http://localhost:5002"

In [ ]:
# However docker needs to address itself internally using its hostname and the normal port

In [ ]:
DOCKER_NETWORK_URL = "http://network:5000"
DOCKER_DOMAIN_URL = "http://domain:5000"

In [ ]:
HOST_MAPPINGS[NETWORK_URL] = DOCKER_NETWORK_URL
HOST_MAPPINGS[DOMAIN_URL] = DOCKER_DOMAIN_URL

In [ ]:
INV_HOST_MAPPINGS = {value: key for (key, value) in HOST_MAPPINGS.items()}

In [ ]:
INV_HOST_MAPPINGS

In [ ]:
do_email, do_pw = "owner@openmined.org", "12345"
ds_email, ds_pw = "data_scientist@email.com", "data_scientist_pwd123"
net_email, net_pw = "network@mymail.com", "network_pw"
token = "9G9MJ06OQH"
network_allowlist = [DOMAIN_URL]

# Utils

In [ ]:
def setup_and_connect(
    url, email, pw, token, node_name="My Node", domain_name="Openmined Domain"
):
    def _connect():
        return connect(url=url, credentials={"email": email, "password": pw})

    try:
        return _connect()
    except Exception as e:
        client = connect(url=url)
        client.setup(
            email=email,
            password=pw,
            node_name=node_name,
            domain_name=domain_name,
            token=token,
        )
        return _connect()

In [ ]:
def is_associated(client, network_url):
    requests = client.association_requests.all()
    return any(
        [x["address"] == network_url and x["accepted"] == True for x in requests]
    )

# Setup Network

For networks, there is a small naming issue in pygrid `setup_serice.py` line 108 ("msg" -> "message"), you might have to run this twice to work. 

In [ ]:
network_client = setup_and_connect(
    NETWORK_URL, net_email, net_pw, token, domain_name="Network"
)

# Data owner: setup, connect to network, and load data

## Setup

In [ ]:
do_client = setup_and_connect(DOMAIN_URL, do_email, do_pw, token)

In [ ]:
do_client.users.all()

## Association Request

In [ ]:
if not is_associated(do_client, HOST_MAPPINGS[NETWORK_URL]):
    do_client.association_requests.create(
        name="My request",
        address=HOST_MAPPINGS[NETWORK_URL],
        sender_address=HOST_MAPPINGS[DOMAIN_URL],
    )

### Network Accepts requests from allowlisted URLS

In [ ]:
# add the internal hosts to the allowlist

In [ ]:
network_allowlist.append(DOCKER_NETWORK_URL)
network_allowlist.append(DOCKER_DOMAIN_URL)

In [ ]:
for req in network_client.association_requests.all():
    if req["address"] in network_allowlist and req["accepted"] == False:
        network_client.association_requests[req["id"]].accept();

In [ ]:
network_client.association_requests.all(pandas=True)

## Create dataset (DO)

In [ ]:
tag = "#ages"

In [ ]:
data_x = th.Tensor([28, 30, 31, 40, 55, 26, 36])

In [ ]:
x_ptr = data_x.send(do_client, pointable=True, tags=[f"{tag}:x"])

## Create DS account (DO)

If we do this for multiple `Domains` we need to make sure that we dont use the same password for different domains. As this may leak access keys between domains

In [ ]:
if not any([x["email"] == ds_email for x in do_client.users.all()]):
    do_client.users.create(email=ds_email, password=ds_pw)

# Data scientist: search & train

## Search

TODO: this should ideally be a separate client (with user permissions) in the future. For now this is oke as we assume that the network owner and the data scientist are from the same org.

In [ ]:
query = f"{tag}:x"

In [ ]:
network_client.search(query=[query], pandas=True)

In [ ]:
urls = network_client.search(query=[query])["match-nodes"]
url = urls[0]
url

In [ ]:
url = INV_HOST_MAPPINGS[url]
url

## Connect to domain

In [ ]:
ds_client = connect(url=url, credentials={"email": ds_email, "password": ds_pw})

In [ ]:
ds_client.store.pandas

In [ ]:
ages_mean_ptr = ds_client.store["#ages:x"].mean()

In [ ]:
ds_client.store.pandas

In [ ]:
ages_mean_ptr.request(reason="Can I have it please?")

In [ ]:
do_client.requests.pandas

In [ ]:
do_client.requests[-1].approve()

In [ ]:
nb_mean = data_x.mean()
remote_mean = ages_mean_ptr.get(delete_obj=False)

In [ ]:
nb_mean, remote_mean